# Import the required libraries <a class="anchor" id="import_libraries"></a>

In [ ]:
%matplotlib widget
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import atomap.api as am
hs.preferences.GUIs.warn_if_guis_are_missing = False
hs.preferences.save()
plt.rcParams['figure.figsize'] = (7,7)

# Open the file <a class="anchor" id="open_file"></a>
The file needs to be at the same directory as the notebook.

Supported formats:
- .dm3
- .dm4
- .tif
- .jpg
- .png
-https://hyperspy.readthedocs.io/en/stable/user_guide/io.html#supported-formats

In [ ]:
s=hs.load('qw1.dm4')
s.plot()
s

## Axes scaling (pixel distance) <a class="anchor" id="pixel_distance"></a>
<p style='text-align: right;'> $Optional$ </p>

It is not necessary to set the scale and center of the image because the processing uses the size in pixels but it is recommended to get a better visualization (.dm3 and .dm4 import settings, so it is not necessary the scale).

To uncomment the cell select the text and press **`Ctrl + /`**
> **`y_size_nm`**: height of the image in nanometers.
<br>
    >The width of the image in nanometers is not necessary because an aspect ratio of 1 is assumed.
    <br>
> **`x_offset_nm/y_offset_nm`**: origin in nanometers.

In [ ]:
# y_size_nm = 10
# x_offset_nm = 0
# y_offset_nm = 0

# s.axes_manager[0].name = 'X'
# s.axes_manager[1].name = 'Y'
# img_scale = y_size_nm/s.data.shape[0]
# s.axes_manager['X'].scale = img_scale
# s.axes_manager['Y'].scale = img_scale
# s.axes_manager['X'].units = 'nm'
# s.axes_manager['Y'].units = 'nm'
# s.axes_manager['X'].offset = x_offset_nm
# s.axes_manager['Y'].offset = y_offset_nm
# s.metadata.General.title = 'Quantum Well'
# s.plot()

## Intensity scaling (pixel intensity) <a class="anchor" id="pixel_intensity"></a>
<p style='text-align: right;'> $Optional$ </p>

If the image has been previously normalized to the impinging beam but the imported image has not the correct scale, we can normalize it with the minimum and maximum values.
<br>
To uncomment the cell select the text and press **`Ctrl + /`**

In [ ]:
# min_intensity=0.055
# max_intensity=0.099

# s.data=s.data=s.data.astype('float32')
# s.data=s.data-np.min(s.data)
# s.data=s.data*(max_intensity-min_intensity)/np.max(s.data)
# s.data=s.data+min_intensity
# s.plot()

### Normalization between 0 and 1
<p style='text-align: right;'> $Optional$ </p>

If we don't have the image normalized to the impinging beam, we still can process the image but with a relative scale between 0 and 1.
<br>
To uncomment the cell select the text and press **`Ctrl + /`**

In [ ]:
# from atomap.atom_finding_refining import normalize_signal
# axes_scale=s.axes_manager
# s = normalize_signal(s)
# s.axes_manager=axes_scale
# s.plot()

# Get the sublattices <a class="anchor" id="sublattices"></a>

Atomap allow us to obtain each sublattice based on the intensity and the separation of the atom columns, so it is hard to select the III or V sublattice due to the intensity variation along the quantum well. Therefore, in this specific case (III/V semiconductors) we can select all the atom columns and then sort the odd (first sublattice) and even (second sublattice) layers in an specific direction.

## Step 1: Get the whole lattice <a class="anchor" id="sublattices_s1"></a>

Varying **`optimal_separation`**  we can see how could be the recognized lattice (or sublattice). This function uses the peak finding algorithm, so the brighter regions (atom columns) are recognized.

In [ ]:
optimal_separation=4

atom_positions = am.get_atom_positions(s, optimal_separation,pca=True)
sublattice = am.Sublattice(atom_positions, s)
sublattice.get_atom_list_on_image(markersize=10).plot()
ax = plt.gca()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

Once we have found the optimal separation we can add or remove the missing atom columns with the **`add_atoms_with_gui`** function. Just click on the atom to add or remove.

In [ ]:
atom_positions_new = am.add_atoms_with_gui(s, atom_positions)

With the intensity peaks found, we proceed to get the whole lattice (size and position of every atom column) using refinement functions.

In [ ]:
sublattice = am.Sublattice(atom_positions_new, s)
sublattice.find_nearest_neighbors()
sublattice.refine_atom_positions_using_center_of_mass()
sublattice.refine_atom_positions_using_2d_gaussian()
sublattice

Plotting:

In [ ]:
sublattice.plot(markersize=10)
ax = plt.gca()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

## Step 2: Separate into III and V sublattices <a class="anchor" id="sublattices_s2"></a>

In order to separate the two sublattices, we need to select the odd and even layers in a certain direction. This direction is chosen in agreement with the corresponding zone axis.

First, the zone axes are constructed:

In [ ]:
sublattice.construct_zone_axes(atom_plane_tolerance=0.8)

To select the optimal zone axis, the parameter **`direction`** is varied until the optimum is found.

In [ ]:
direction=3

zone_vector = sublattice.zones_axis_average_distances[direction]
atom_planes = sublattice.atom_planes_by_zone_vector[zone_vector]
zone_axis = sublattice.get_atom_planes_on_image(atom_planes)
zone_axis.plot()
ax = plt.gca()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

Once the optimal zone axis has been found, the planes are classified into odd and even layers. Next, we construct two lists with the atom column positions of the two sublattices.

In [ ]:
zone_axis = sublattice.zones_axis_average_distances[direction]
atom_plane_list = sublattice.atom_planes_by_zone_vector[zone_axis]
odd_i = [] ; even_i = []

for i in range(0, len(atom_plane_list)):
    if i % 2:
        even_i.append(atom_plane_list[i])
    else :
        odd_i.append(atom_plane_list[i])
        
x=[] ; y=[]
for i in range(0, len(odd_i)):
    x=x+odd_i[i].x_position
    y=y+odd_i[i].y_position
atom_pos_odd=np.stack((np.array(x),np.array(y)),axis=1) #atom positions of the odd layers
x=[];y=[]
for i in range(0, len(even_i)):
    x=x+even_i[i].x_position
    y=y+even_i[i].y_position
atom_pos_even=np.stack((np.array(x),np.array(y)),axis=1) #atom positions of the even layers

Finally, with the atom column positions, the two sublattices are constructed and refined.

In [ ]:
sublattice_A = am.Sublattice(atom_pos_odd, image=s)
sublattice_B = am.Sublattice(atom_pos_even, image=s,color='blue')
sublattice_A.find_nearest_neighbors()
sublattice_A.refine_atom_positions_using_center_of_mass()
sublattice_A.refine_atom_positions_using_2d_gaussian()
sublattice_B.find_nearest_neighbors()
sublattice_B.refine_atom_positions_using_center_of_mass()
sublattice_B.refine_atom_positions_using_2d_gaussian()

Plotting:

In [ ]:
atom_lattice = am.Atom_Lattice(s, name='quantum_well', sublattice_list=[sublattice_A, sublattice_B])
atom_lattice.plot(markersize=10)
ax = plt.gca()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

Before  obtaining the intensity maps, we can save the atom lattice in a .hdf5 file:

In [ ]:
s.save('data.hspy')
atom_lattice.save('atom_lattice_qw.hdf5', overwrite=True)